# [Day 8: Seven Segment Search](https://adventofcode.com/2021/day/8)

In [1]:
import collections as cl

## Part 1

In [2]:
example_data = [
    "be cfbegad cbdgef fgaecd cgeb fdcge agebfd fecdb fabcd edb | fdgacbe cefdb cefbgd gcbe",
    "edbfga begcd cbg gc gcadebf fbgde acbgfd abcde gfcbed gfec | fcgedb cgb dgebacf gc",
    "fgaebd cg bdaec gdafb agbcfd gdcbef bgcad gfac gcb cdgabef | cg cg fdcagb cbg",
    "fbegcd cbd adcefb dageb afcb bc aefdc ecdab fgdeca fcdbega | efabcd cedba gadfec cb",
    "aecbfdg fbg gf bafeg dbefa fcge gcbea fcaegb dgceab fcbdga | gecf egdcabf bgf bfgea",
    "fgeab ca afcebg bdacfeg cfaedg gcfdb baec bfadeg bafgc acf | gebdcfa ecba ca fadegcb",
    "dbcfg fgd bdegcaf fgec aegbdf ecdfab fbedc dacgb gdcebf gf | cefg dcbef fcge gbcadfe",
    "bdfegc cbegaf gecbf dfcage bdacg ed bedf ced adcbefg gebcd | ed bcgafe cdgba cbgef",
    "egadfb cdbfeg cegd fecab cgb gbdefca cg fgcdab egfdb bfceg | gbdfcae bgc cg cgb",
    "gcafb gcf dcaebfg ecagb gf abcdeg gaef cafbge fdbac fegbdc | fgae cfgab fg bagce",
]

In [3]:
segment_to_digit = {
    "abcefg": 0,
    "cf": 1,
    "acdeg": 2,
    "acdfg": 3,
    "bcdf": 4,
    "abdfg": 5,
    "abdefg": 6,
    "acf": 7,
    "abcdefg": 8,
    "abcdfg": 9,
}

pattern_lengths = {segment_to_digit[pattern]: len(pattern) for pattern, n in segment_to_digit.items()}
digits_by_length = cl.defaultdict(list)
for digit, length in pattern_lengths.items():
    digits_by_length[length].append(digit)
unique_lengths = [digits[0] for length, digits in digits_by_length.items() if len(digits) == 1]

print(f"Unique lengths: {', '.join([str(d) for d in sorted(unique_lengths)])}")

Unique lengths: 1, 4, 7, 8


In [4]:
def parse_line(line):
    patterns, digits = map(str.strip, line.split("|"))
    return {"patterns": patterns.split(" "), "digits": digits.split(" ") }

In [5]:
def part1(input_data):
    cnt = cl.Counter()
    for line in input_data:
        cnt.update(map(len, parse_line(line)["digits"]))
    return sum([cnt[pattern_lengths[n]] for n in unique_lengths])

In [6]:
print(f"Check part 1: {part1(example_data) == 26}")

Check part 1: True


In [7]:
with open(r"..\data\Day 08 input.txt", "r") as fh_in:
    input_data = fh_in.readlines()
print(f"Input check: {len(input_data) == 200}")

Input check: True


In [8]:
print(f"Answer part 1: {part1(input_data)}")

Answer part 1: 392


## Part 2

In [9]:
def decode(line, segment_to_digit=segment_to_digit):
    parsed_line = parse_line(line)
    
    # create list of patterns sorted by length
    patterns_by_len = cl.defaultdict(list)
    for pattern in parsed_line["patterns"]:
        patterns_by_len[len(pattern)].append(set(pattern))
    
    # combine patterns to extract segment information
    segments = {}
    segments["a"] = patterns_by_len[3][0] - patterns_by_len[2][0]
    segments_bd = patterns_by_len[4][0] - patterns_by_len[3][0]
    segments_eg = patterns_by_len[7][0] - patterns_by_len[4][0] - segments["a"]
    segments_afg = sorted([five - segments_bd for five in patterns_by_len[5]], key=len)[0]
    segments["f"] = segments_afg - segments["a"] - segments_eg
    segments["g"] = segments_afg - segments["a"] - segments["f"]
    segments["e"] = segments_eg - segments["g"]
    segments["c"] = sorted([five - segments_bd - segments_afg for five in patterns_by_len[5]], key=len)[1]
    segments["b"] = sorted(
        [six - segments_eg - segments["f"] - segments["a"] - segments["c"] for six in patterns_by_len[6]],
        key=len
    )[0]
    segments["d"] = segments_bd - segments["b"]
    
    # decode by remapping and lookup
    mapping = str.maketrans({list(segments[to_digit])[0]: to_digit for to_digit in segments})
    decoded = [segment_to_digit["".join(sorted(digit.translate(mapping)))] for digit in parsed_line["digits"]]
    return int("".join(map(str, decoded)))

In [10]:
example_line = "acedgfb cdfbe gcdfa fbcad dab cefabd cdfgeb eafb cagedb ab | cdfeb fcadb cdfeb cdbaf"
print(f"Check part 2 (line): {decode(example_line) == 5353}")

Check part 2 (line): True


In [11]:
example_answers = {
    "be cfbegad cbdgef fgaecd cgeb fdcge agebfd fecdb fabcd edb | fdgacbe cefdb cefbgd gcbe": 8394,
    "edbfga begcd cbg gc gcadebf fbgde acbgfd abcde gfcbed gfec | fcgedb cgb dgebacf gc": 9781,
    "fgaebd cg bdaec gdafb agbcfd gdcbef bgcad gfac gcb cdgabef | cg cg fdcagb cbg": 1197,
    "fbegcd cbd adcefb dageb afcb bc aefdc ecdab fgdeca fcdbega | efabcd cedba gadfec cb": 9361,
    "aecbfdg fbg gf bafeg dbefa fcge gcbea fcaegb dgceab fcbdga | gecf egdcabf bgf bfgea": 4873,
    "fgeab ca afcebg bdacfeg cfaedg gcfdb baec bfadeg bafgc acf | gebdcfa ecba ca fadegcb": 8418,
    "dbcfg fgd bdegcaf fgec aegbdf ecdfab fbedc dacgb gdcebf gf | cefg dcbef fcge gbcadfe": 4548,
    "bdfegc cbegaf gecbf dfcage bdacg ed bedf ced adcbefg gebcd | ed bcgafe cdgba cbgef": 1625,
    "egadfb cdbfeg cegd fecab cgb gbdefca cg fgcdab egfdb bfceg | gbdfcae bgc cg cgb": 8717,
    "gcafb gcf dcaebfg ecagb gf abcdeg gaef cafbge fdbac fegbdc | fgae cfgab fg bagce": 4315,
}

for line, answer in example_answers.items():
    print(f"{line}: {decode(line) == answer}")

be cfbegad cbdgef fgaecd cgeb fdcge agebfd fecdb fabcd edb | fdgacbe cefdb cefbgd gcbe: True
edbfga begcd cbg gc gcadebf fbgde acbgfd abcde gfcbed gfec | fcgedb cgb dgebacf gc: True
fgaebd cg bdaec gdafb agbcfd gdcbef bgcad gfac gcb cdgabef | cg cg fdcagb cbg: True
fbegcd cbd adcefb dageb afcb bc aefdc ecdab fgdeca fcdbega | efabcd cedba gadfec cb: True
aecbfdg fbg gf bafeg dbefa fcge gcbea fcaegb dgceab fcbdga | gecf egdcabf bgf bfgea: True
fgeab ca afcebg bdacfeg cfaedg gcfdb baec bfadeg bafgc acf | gebdcfa ecba ca fadegcb: True
dbcfg fgd bdegcaf fgec aegbdf ecdfab fbedc dacgb gdcebf gf | cefg dcbef fcge gbcadfe: True
bdfegc cbegaf gecbf dfcage bdacg ed bedf ced adcbefg gebcd | ed bcgafe cdgba cbgef: True
egadfb cdbfeg cegd fecab cgb gbdefca cg fgcdab egfdb bfceg | gbdfcae bgc cg cgb: True
gcafb gcf dcaebfg ecagb gf abcdeg gaef cafbge fdbac fegbdc | fgae cfgab fg bagce: True


In [12]:
print(f"Answer part 2: {sum([decode(line) for line in input_data])}")

Answer part 2: 1004688
